# PROJETO IMOVEIS
---

#**0. Instalação e importação das bibliotecas**
---

## Configurações do projeto

### Importando pacotes

In [1]:
!pip install gcsfs 
!pip install pyspark
!{sys.executable} -m pip install cloud-sql-python-connector["pymysql"] SQLAlchemy
!pip install pymongo[srv]
!pip install gcsfs


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
/bin/bash: {sys.executable}: command not found
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
#import Bibliotecas
import pandas as pd
import numpy as np
import json
import os
from pyspark.sql import SparkSession
from google.cloud import storage

# Pandas
import pandas as pd

# Conexão do Storage
from google.cloud import storage
import os

# Conector do Mysql
import sys
from sqlalchemy import create_engine
import sqlalchemy


###Conexões das bibliotecas

In [3]:
#Conexão com o drive 
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
#Conecção com a GCP
serviceAccount = '/content/drive/MyDrive/BC26-ENG DADOS-PYTHON/PROJETOS/PROJETO ESTUDO/Cópia de key-store-via-colab.json'
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = serviceAccount

In [5]:
#CONFIGURAR A SPARK SESSION
spark = (
   SparkSession.builder
               .master('local')
               .appName('gcsfs') 
               .config('spark.ui.port', '4050')
               .config("spark.jars", 'https://storage.googleapis.com/hadoop-lib/gcs/gcs-connector-hadoop2-latest.jar') #config acesso conector gcp
               .getOrCreate()                
)

### Opções de configuração

In [6]:
#Para mostrar todas as colunas no display
pd.set_option('display.max_columns',25)

In [7]:
pd.get_option('display.max_rows', 1000)

60

#**1. Carregando os dados**

### Arquivos JSON - `read_json`

In [8]:
dados = pd.read_json(
    path_or_buf='gs://projeto_cat/ORIGINAL/realestates.json',
    orient='columns'
)
dados

,normal,highlights
output,{'listings': [{'imovel': {'tipos': {'proprieda...,{'listings': [{'imovel': {'tipos': {'proprieda...


### Arquivos EXCEL - `read_excel`

In [9]:
bairros = pd.read_excel(
    io = 'gs://projeto_cat/ORIGINAL/bairros.xlsx',
    sheet_name='Preço médio por tipo',
    usecols="C:E",
    header = 2,
    index_col=[0, 1],
    names=['bairros','tipo','valor_m2_bairro']
)
bairros

valor_m2_bairro
bairros       tipo                             
São Cristóvão Casa                        13793
              Apartamento                  6306
              Cobertura                   11695
              Consultório                  6991
              Imóvel Comercial            19633
...                                         ...
Vista Alegre  Cobertura                   16567
              Consultório                  5812
              Imóvel Comercial            12895
              Loja                         6182
              Sala Comercial               3377

[1127 rows x 1 columns]

'# Carregando os dados BRUTOS para o mongo DB

In [10]:
# #Criar conexão
# db = client['IMOVEIS']

In [11]:
# # #Criar e enviar coleção ORIGINAL  
# colecao_1 = db['realestates-ORIGINAL']
# df1_dict = dados.to_dict('records')
# # Inserindo os dados no mongodb
# colecao_1.insert_many(df1_dict)

In [12]:
# colecao_2_parte1 = db['bairros-ORIGINAL']
# df2_dict = bairros.to_dict('records')
# colecao_2_parte1.insert_many(df2_dict)

#**2. Tratamento**

## TRANSFORMANDO E TRATANDO OS DADOS
---

### Transformando dados no formato JSON para uma tabela
- Utilizando o json_normalize para informações em colunas 
- existem duas colunas inicialmente nesse dataframe Coluna Normal e Highlights

In [13]:
#Transformar essas informações em linhas da minha tabela , ele vai utilizar as informações conditas para nomes das colunas 
dados_normal_listings = pd.json_normalize(data=dados.normal, sep='_', record_path=['listings'])

In [14]:
dados_highlights_listings = pd.json_normalize(data=dados.highlights, sep='_', record_path=['listings'])
dados_highlights_listings

,imovel_tipos_propriedade,imovel_endereco_bairro,imovel_endereco_localizacao_type,imovel_endereco_localizacao_coordinates,imovel_vagasGaragem,imovel_area,imovel_caracteristicas_propriedade,imovel_caracteristicas_condominio,imovel_caracteristicas_entorno,anuncio_tipos_publicacao,anuncio_tipos_listagem,anuncio_valores_venda,anuncio_valores_aluguel,anuncio_valores_condominio,anuncio_valores_iptu,anuncio_descricao
0,Casa,Engenho Novo,Point,"[-43.2634918, -22.8971768]",0,0,"['Sauna', 'Terraço', 'Acessibilidade', 'Mobili...","['Estacionamento Coberto', 'Estacionamento Vis...","['Próximo a Serviços de Saúde', 'Próximo ao Sh...",Padrão,Usado,195772.0,0.0,0.0,0.0,"Amplo imóvel para venda com 0 quartos, sendo 0..."
1,Cobertura,Vargem Grande,Point,"[-43.4907917, -22.988122]",1,61,[],"['Estacionamento Visitantes', 'Estacionamento ...","['Próximo ao Metrô', 'Próximo a Praia', 'Próxi...",Destaque,Usado,305000.0,0.0,667.0,162.0,"Amplo imóvel para venda com 2 quartos, sendo 0..."
2,Imóvel Comercial,Ribeira,Point,"[-43.1690005, -22.8239272]",4,180,"['Vista Panorâmica', 'Acessibilidade', 'Área d...","['Segurança Interna', 'Salão de Festas', 'Acad...","['Próximo a Restaurante', 'Próximo a Praia', '...",Padrão,Usado,1050000.0,0.0,0.0,315.0,"Amplo imóvel para venda com 3 quartos, sendo 1..."
3,Apartamento,Praça Seca,Point,"[0, 0]",1,50,"['Sauna', 'Quarto/WC Empregada']","['Salão de Jogos', 'Segurança Interna', 'Estac...","['Próximo ao Shopping', 'Próximo a Escola Part...",Padrão,Usado,250000.0,0.0,350.0,150.0,"Amplo imóvel para venda com 2 quartos, sendo 0..."
4,Apartamento,Honório Gurgel,Point,"[-43.3614243, -22.8416031]",0,148,"['Terraço', 'Piscina', 'Gás Encanado', 'Acessi...","['Sauna', 'Bicicletário', 'Academia', 'Serviço...","['Próximo a Serviços de Saúde', 'Próximo a Far...",Destaque,Lançamento,165000.0,0.0,270.0,0.0,"Amplo imóvel para venda com 2 quartos, sendo 0..."
5,Apartamento,Barra da Tijuca,Point,"[-43.3245375, -23.006984]",1,79,"['Terraço', 'Acessibilidade', 'Armários Planej...","['Salão de Festas', 'Estacionamento Visitantes...","['Próximo a Mercado', 'Próximo a Parques e Áre...",Premium,Usado,900000.0,0.0,1400.0,180.0,"Amplo imóvel para venda com 3 quartos, sendo 1..."
6,Consultório,Tijuca,Point,"[-43.2558939, -22.9444082]",1,126,"['Armários Planejados', 'Closet', 'Gás Encanad...","['Bicicletário', 'Salão de Jogos', 'Segurança ...","['Próximo a Praia', 'Próximo a Serviços de Saú...",Padrão,Usado,630000.0,0.0,0.0,1500.0,"Amplo imóvel para venda com 2 quartos, sendo 2..."
7,Apartamento,Recreio dos Bandeirantes,Point,"[-43.480981, -23.0160348]",1,126,"['Sauna', 'Cozinha Americana', 'Terraço', 'Gás...","['Estacionamento Visitantes', 'Salão de Festas...","['Próximo a Escola Pública', 'Próximo ao Metrô...",Padrão,Usado,475000.0,0.0,920.0,0.0,"Amplo imóvel para venda com 2 quartos, sendo 1..."
8,Apartamento,Tijuca,Point,"[-43.2266348, -22.9220052]",0,0,"['Ambientes Integrados', 'Varanda', 'Terraço',...","['Elevador', 'Acessibilidade', 'Estacionamento...",[],Padrão,Usado,0.0,0.0,0.0,0.0,"Amplo imóvel para venda com 0 quartos, sendo 0..."
9,Apartamento,Lagoa,Point,"[-43.2112318, -22.9625433]",1,56,"['Sauna', 'Piscina', 'Cozinha Americana', 'Vis...","['Quadra Poliesportiva', 'Bicicletário', 'Pisc...","['Próximo a Farmácia', 'Próximo a Áreas de Laz...",Padrão,Usado,1200000.0,0.0,1608.0,3284.0,"Amplo imóvel para venda com 1 quartos, sendo 1..."


### Trabalhando com dados textuais
- Foi ferificado que em algumas colunas que existem mais de uma informações aparentimente seriam lista mas ao verificar utlizando o type foi visto que se tratavam de str e foi feito esse tratamento em todas as colunas que se encaixam
- Tranformar os dados de str em formato lista

In [15]:
#usando um for para aplicar em todas as colunas
#Usando o filtro para separar as colunas com a mesma quetão de str no lugar de lista - usar o columns apenas para saber o nome 
columns = dados_normal_listings.filter(like='imovel_caracteristicas').columns

for columns in columns:
  dados_normal_listings[columns] = dados_normal_listings[columns].str[1:-1].str.replace("'", "").str.split(", ")
  dados_highlights_listings[columns] = dados_highlights_listings[columns].str[1:-1].str.replace("'", "").str.split(", ")

In [16]:
#ANALISE --Selecionando a primeira linha e o terceiro item da coluna especificada 
dados_normal_listings.loc[0, 'imovel_caracteristicas_propriedade'][2]

'Churrasqueira'

##  COMBINANDO CONJUNTOS DE DADOS
---

### Empilhando *DataFrames*

In [17]:
#criar um novo df Junção dos dois df criados utilizando o metodo concat e ignorando o index 
dados_listings = pd.concat([dados_normal_listings, dados_highlights_listings], ignore_index=True)
dados_listings

,imovel_tipos_propriedade,imovel_endereco_bairro,imovel_endereco_localizacao_type,imovel_endereco_localizacao_coordinates,imovel_vagasGaragem,imovel_area,imovel_caracteristicas_propriedade,imovel_caracteristicas_condominio,imovel_caracteristicas_entorno,anuncio_tipos_publicacao,anuncio_tipos_listagem,anuncio_valores_venda,anuncio_valores_aluguel,anuncio_valores_condominio,anuncio_valores_iptu,anuncio_descricao
0,Casa,Barra da Tijuca,Point,"[-43.3039086, -23.0139692]",4,279,"[Área de Serviço, Terraço, Churrasqueira, Quar...","[Churrasqueira, Salão de Jogos, Sauna, Estacio...","[Próximo a Escola Particular, Próximo ao Metrô...",Padrão,Lançamento,2400000.0,0.0,790.0,5700.0,"Amplo imóvel para venda com 3 quartos, sendo 1..."
1,Apartamento,Campo Grande,Point,"[0, 0]",1,60,[Sauna],"[Elevador, Playground, Salão de Festas, Sistem...","[Próximo ao Shopping, Próximo a Áreas de Lazer...",Padrão,Usado,200000.0,0.0,0.0,0.0,"Amplo imóvel para venda com 2 quartos, sendo 0..."
2,Cobertura,Barra da Tijuca,Point,"[-43.3037186, -22.9951304]",2,380,"[Acessibilidade, Área de Serviço, Sauna, Ambie...","[Sauna, Playground, Quadra Poliesportiva, Salã...",[Próximo a Escola Particular],Premium,Usado,4400000.0,0.0,1200.0,850.0,"Amplo imóvel para venda com 5 quartos, sendo 4..."
3,Cobertura,Barra da Tijuca,Point,"[-43.3548121, -23.0097423]",1,73,"[Vista Panorâmica, Piscina, Sauna, Closet, Mob...","[Quadra Poliesportiva, Aceita Pet, Playground,...","[Próximo a Farmácia, Próximo a Academia, Próxi...",Padrão,Usado,1100000.0,0.0,1300.0,364.0,"Amplo imóvel para venda com 2 quartos, sendo 1..."
4,Sala Comercial,Glória,Point,"[-43.1779703, -22.9174894]",1,32,[Armários Planejados],"[Quadra Poliesportiva, Porteiro 24 Horas, Elev...","[Próximo a Centro Empresarial, Próximo a Áreas...",Padrão,Usado,260000.0,0.0,450.0,800.0,"Amplo imóvel para venda com 0 quartos, sendo 0..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
65,Apartamento,Copacabana,Point,"[-43.1943902, -22.9816847]",1,216,"[Cozinha Americana, Quarto/WC Empregada, Pisci...","[Elevador, Salão de Festas, Porteiro 24 Horas,...","[Próximo a Farmácia, Próximo a Academia, Próxi...",Padrão,Usado,1800000.0,0.0,2000.0,2300.0,"Amplo imóvel para venda com 4 quartos, sendo 2..."
66,Apartamento,Leblon,Point,"[-43.2253114, -22.9843603]",1,218,"[Piscina, Gás Encanado]","[Academia, Aceita Pet, Sistema de Incêndio, Bi...","[Próximo a Comércio Diversos, Próximo a Centro...",Premium,Usado,4500000.0,0.0,3600.0,7000.0,"Amplo imóvel para venda com 3 quartos, sendo 1..."
67,Casa,Copacabana,Point,"[-43.1951842, -22.9818646]",0,25,"[Closet, Armários Planejados, Quarto/WC Empreg...","[Bicicletário, Sistema de Incêndio, Segurança ...",[],Destaque,Usado,350000.0,0.0,290.0,182.0,"Amplo imóvel para venda com 1 quartos, sendo 0..."
68,Sala Comercial,Cachambi,Point,"[-43.2781994, -22.8917611]",2,138,"[Gás Encanado, Acessibilidade, Ambientes Integ...","[Playground, Depósito Privativo, Academia, Qua...","[Próximo a Praia, Próximo a Serviços de Saúde,...",Premium,Usado,619000.0,0.0,506.0,1404.0,"Amplo imóvel para venda com 3 quartos, sendo 1..."


### Combinando *DataFrames* a partir de uma chave de ligação

#### Tratando a variável de ligação

##### Selecionando a variável de ligação nos dois *DataFrames*

In [18]:
#Usando esse metodo get_level_values conseguimos isolar esses index dp df bairros e transformar em uma lista 
bairros_todos = bairros.index.get_level_values('bairros')
#Com essa lista de baiiros vamos criar uma series utilizando apenas os nomes unicos 
bairros_todos = pd.Series(bairros_todos.unique())
bairros_todos 

0            São Cristóvão
1                  Benfica
2                     Caju
3                  Catumbi
4                   Centro
              ...         
156    Vicente de Carvalho
157          Vigário Geral
158          Vila da Penha
159            Vila Kosmos
160           Vista Alegre
Name: bairros, Length: 161, dtype: object

In [19]:
#Replece onde é necessário
dados_listings['imovel_endereco_bairro'].replace('Freguesia(Jacarepaguá)', 'Frequesia', inplace=True)

#### Combinando os *DataFrames* - usando a função [merge](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.merge.html)

In [20]:
dados_listings = pd.merge(
    left= dados_listings,
    right=bairros,
    left_on=['imovel_endereco_bairro', 'imovel_tipos_propriedade'],  #colunas de ligação
    right_index= True   #usando o index
)

dados_listings

,imovel_tipos_propriedade,imovel_endereco_bairro,imovel_endereco_localizacao_type,imovel_endereco_localizacao_coordinates,imovel_vagasGaragem,imovel_area,imovel_caracteristicas_propriedade,imovel_caracteristicas_condominio,imovel_caracteristicas_entorno,anuncio_tipos_publicacao,anuncio_tipos_listagem,anuncio_valores_venda,anuncio_valores_aluguel,anuncio_valores_condominio,anuncio_valores_iptu,anuncio_descricao,valor_m2_bairro
0,Casa,Barra da Tijuca,Point,"[-43.3039086, -23.0139692]",4,279,"[Área de Serviço, Terraço, Churrasqueira, Quar...","[Churrasqueira, Salão de Jogos, Sauna, Estacio...","[Próximo a Escola Particular, Próximo ao Metrô...",Padrão,Lançamento,2400000.0,0.0,790.0,5700.0,"Amplo imóvel para venda com 3 quartos, sendo 1...",12835
47,Casa,Barra da Tijuca,Point,"[-43.3413206, -22.99961]",0,370,"[Acessibilidade, Quarto/WC Empregada, Armários...","[Churrasqueira, Depósito Privativo, Quadra Pol...","[Próximo a Restaurante, Próximo a Comércio Div...",Super Destaque,Usado,3200000.0,0.0,2520.0,7600.0,"Amplo imóvel para venda com 4 quartos, sendo 1...",12835
1,Apartamento,Campo Grande,Point,"[0, 0]",1,60,[Sauna],"[Elevador, Playground, Salão de Festas, Sistem...","[Próximo ao Shopping, Próximo a Áreas de Lazer...",Padrão,Usado,200000.0,0.0,0.0,0.0,"Amplo imóvel para venda com 2 quartos, sendo 0...",6425
16,Apartamento,Campo Grande,Point,"[-43.1728965, -22.9068467]",0,40,"[Vista Panorâmica, Sauna, Área de Serviço, Pis...","[Bicicletário, Comércio/Serviços no Condomínio...","[Próximo a Escola Particular, Próximo a Praia,...",Padrão,Usado,700000.0,0.0,0.0,0.0,"Amplo imóvel para venda com 2 quartos, sendo 0...",6425
2,Cobertura,Barra da Tijuca,Point,"[-43.3037186, -22.9951304]",2,380,"[Acessibilidade, Área de Serviço, Sauna, Ambie...","[Sauna, Playground, Quadra Poliesportiva, Salã...",[Próximo a Escola Particular],Premium,Usado,4400000.0,0.0,1200.0,850.0,"Amplo imóvel para venda com 5 quartos, sendo 4...",12585
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
60,Apartamento,Vila Isabel,Point,"[-43.2508337, -22.9167328]",1,70,"[Gás Encanado, Quarto/WC Empregada, Terraço, A...",[Academia],"[Próximo a Mercado, Próximo a Praia, Próximo a...",Padrão,Usado,370000.0,0.0,500.0,0.0,"Amplo imóvel para venda com 2 quartos, sendo 0...",5964
63,Cobertura,Méier,Point,"[-43.279874, -22.894531]",0,0,[],"[Playground, Elevador, Salão de Festas, Estaci...","[Próximo a Restaurante, Próximo a Mercado, Pró...",Padrão,Usado,350000.0,0.0,0.0,0.0,"Amplo imóvel para venda com 0 quartos, sendo 0...",17199
64,Cobertura,Méier,Point,"[0, 0]",0,62,"[Terraço, Área de Serviço, Sauna, Churrasqueir...","[Acessibilidade, Bicicletário, Aceita Pet, Chu...","[Próximo ao Metrô, Próximo a Restaurante, Próx...",Padrão,Usado,265000.0,0.0,220.0,0.0,"Amplo imóvel para venda com 3 quartos, sendo 0...",17199
66,Apartamento,Leblon,Point,"[-43.2253114, -22.9843603]",1,218,"[Piscina, Gás Encanado]","[Academia, Aceita Pet, Sistema de Incêndio, Bi...","[Próximo a Comércio Diversos, Próximo a Centro...",Premium,Usado,4500000.0,0.0,3600.0,7000.0,"Amplo imóvel para venda com 3 quartos, sendo 1...",6388


## ADICIONANDO INFORMAÇÕES
---

In [21]:
#Criando as colunas quartos, suites e banheiros
#na coluna anuncio_descricao precisamos extrair esses dados e transformalos em colunas do df para isso seguimos os seguintes passos:
configuracao = dados_listings['anuncio_descricao'].str.extractall('(\d+)')
#para transformar as linhas criadas em colunas utilizamos o metodo unstack 
configuracao= configuracao.unstack().rename(columns={0:'quartos', 1:'suites', 2:'banheiros'})
#utilizar o droplevel para retirar o primeiro nivel 
configuracao = configuracao.droplevel(level=0, axis=1)
#agr que as configurações estão prontas podemos juntar com o DF Principal, utilizando os index para fazer a junção
dados_listings = pd.merge(dados_listings, configuracao, left_index=True, right_index=True)

In [22]:
#Criando colunas a partir das classificações
#Para utilizar o metodo map criamos um dicionario com as classificações 
tipo_uso = {
    'Apartamento': 'Residencial',
    'Casa': 'Residencial',
    'Cobertura': 'Residencial',
    'Consultório': 'Comercial',
    'Imóvel Comercial': 'Comercial',
    'Loja': 'Comercial',
    'Sala Comercial': 'Comercial'
}
#posteriomente adicionamos em uma variavel
imovel_tipos_uso = dados_listings.imovel_tipos_propriedade.map(tipo_uso)
#Inserindo como nova coluna no DF Original usando o metodo insert
dados_listings.insert(loc=0, column='imovel_tipos_uso', value=imovel_tipos_uso)

In [23]:
#Criando colunas a partir da classificação de valores 
#utilizando o metodo cut vamos cria novas classificações rotulos
valor_minimo = dados_listings['anuncio_valores_venda'].min()
valor_maximo = dados_listings['anuncio_valores_venda'].max()
rotulos = ['Popular', 'Padrão', 'Alto']
#É preciso N pontos para obter N - 1 classes
classes = [valor_minimo, 400000, 2000000, valor_maximo]
#com as classes personalizadas eu uso o metodo cut bins parametro de classe que irar ser criadas e labels com os rotulos 
dados_listings['classe_valor'] = pd.cut(x=dados_listings['anuncio_valores_venda'], bins=classes, labels=rotulos, include_lowest=True)
dados_listings

,imovel_tipos_uso,imovel_tipos_propriedade,imovel_endereco_bairro,imovel_endereco_localizacao_type,imovel_endereco_localizacao_coordinates,imovel_vagasGaragem,imovel_area,imovel_caracteristicas_propriedade,imovel_caracteristicas_condominio,imovel_caracteristicas_entorno,anuncio_tipos_publicacao,anuncio_tipos_listagem,anuncio_valores_venda,anuncio_valores_aluguel,anuncio_valores_condominio,anuncio_valores_iptu,anuncio_descricao,valor_m2_bairro,quartos,suites,banheiros,classe_valor
0,Residencial,Casa,Barra da Tijuca,Point,"[-43.3039086, -23.0139692]",4,279,"[Área de Serviço, Terraço, Churrasqueira, Quar...","[Churrasqueira, Salão de Jogos, Sauna, Estacio...","[Próximo a Escola Particular, Próximo ao Metrô...",Padrão,Lançamento,2400000.0,0.0,790.0,5700.0,"Amplo imóvel para venda com 3 quartos, sendo 1...",12835,3,1,2,Alto
47,Residencial,Casa,Barra da Tijuca,Point,"[-43.3413206, -22.99961]",0,370,"[Acessibilidade, Quarto/WC Empregada, Armários...","[Churrasqueira, Depósito Privativo, Quadra Pol...","[Próximo a Restaurante, Próximo a Comércio Div...",Super Destaque,Usado,3200000.0,0.0,2520.0,7600.0,"Amplo imóvel para venda com 4 quartos, sendo 1...",12835,4,1,2,Alto
1,Residencial,Apartamento,Campo Grande,Point,"[0, 0]",1,60,[Sauna],"[Elevador, Playground, Salão de Festas, Sistem...","[Próximo ao Shopping, Próximo a Áreas de Lazer...",Padrão,Usado,200000.0,0.0,0.0,0.0,"Amplo imóvel para venda com 2 quartos, sendo 0...",6425,2,0,1,Popular
16,Residencial,Apartamento,Campo Grande,Point,"[-43.1728965, -22.9068467]",0,40,"[Vista Panorâmica, Sauna, Área de Serviço, Pis...","[Bicicletário, Comércio/Serviços no Condomínio...","[Próximo a Escola Particular, Próximo a Praia,...",Padrão,Usado,700000.0,0.0,0.0,0.0,"Amplo imóvel para venda com 2 quartos, sendo 0...",6425,2,0,1,Padrão
2,Residencial,Cobertura,Barra da Tijuca,Point,"[-43.3037186, -22.9951304]",2,380,"[Acessibilidade, Área de Serviço, Sauna, Ambie...","[Sauna, Playground, Quadra Poliesportiva, Salã...",[Próximo a Escola Particular],Premium,Usado,4400000.0,0.0,1200.0,850.0,"Amplo imóvel para venda com 5 quartos, sendo 4...",12585,5,4,5,Alto
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
60,Residencial,Apartamento,Vila Isabel,Point,"[-43.2508337, -22.9167328]",1,70,"[Gás Encanado, Quarto/WC Empregada, Terraço, A...",[Academia],"[Próximo a Mercado, Próximo a Praia, Próximo a...",Padrão,Usado,370000.0,0.0,500.0,0.0,"Amplo imóvel para venda com 2 quartos, sendo 0...",5964,2,0,1,Popular
63,Residencial,Cobertura,Méier,Point,"[-43.279874, -22.894531]",0,0,[],"[Playground, Elevador, Salão de Festas, Estaci...","[Próximo a Restaurante, Próximo a Mercado, Pró...",Padrão,Usado,350000.0,0.0,0.0,0.0,"Amplo imóvel para venda com 0 quartos, sendo 0...",17199,0,0,1,Popular
64,Residencial,Cobertura,Méier,Point,"[0, 0]",0,62,"[Terraço, Área de Serviço, Sauna, Churrasqueir...","[Acessibilidade, Bicicletário, Aceita Pet, Chu...","[Próximo ao Metrô, Próximo a Restaurante, Próx...",Padrão,Usado,265000.0,0.0,220.0,0.0,"Amplo imóvel para venda com 3 quartos, sendo 0...",17199,3,0,1,Popular
66,Residencial,Apartamento,Leblon,Point,"[-43.2253114, -22.9843603]",1,218,"[Piscina, Gás Encanado]","[Academia, Aceita Pet, Sistema de Incêndio, Bi...","[Próximo a Comércio Diversos, Próximo a Centro...",Premium,Usado,4500000.0,0.0,3600.0,7000.0,"Amplo imóvel para venda com 3 quartos, sendo 1...",6388,3,1,2,Alto


In [24]:
#criando a coluna valor_m2
valor_m2 = lambda data: data['anuncio_valores_venda'] / data['imovel_area'] if data['imovel_area'] != 0 else 0
dados_listings['valor_m2'] = dados_listings.apply(valor_m2, axis=1)
dados_listings[['anuncio_valores_venda', 'imovel_area', 'valor_m2']].iloc[16:19]

,anuncio_valores_venda,imovel_area,valor_m2
8,1390000.0,214,6495.327103
13,370583.0,0,0.000000
24,1600000.0,300,5333.333333


In [25]:
dados_listings.head(1)

,imovel_tipos_uso,imovel_tipos_propriedade,imovel_endereco_bairro,imovel_endereco_localizacao_type,imovel_endereco_localizacao_coordinates,imovel_vagasGaragem,imovel_area,imovel_caracteristicas_propriedade,imovel_caracteristicas_condominio,imovel_caracteristicas_entorno,anuncio_tipos_publicacao,anuncio_tipos_listagem,anuncio_valores_venda,anuncio_valores_aluguel,anuncio_valores_condominio,anuncio_valores_iptu,anuncio_descricao,valor_m2_bairro,quartos,suites,banheiros,classe_valor,valor_m2
0,Residencial,Casa,Barra da Tijuca,Point,"[-43.3039086, -23.0139692]",4,279,"[Área de Serviço, Terraço, Churrasqueira, Quar...","[Churrasqueira, Salão de Jogos, Sauna, Estacio...","[Próximo a Escola Particular, Próximo ao Metrô...",Padrão,Lançamento,2400000.0,0.0,790.0,5700.0,"Amplo imóvel para venda com 3 quartos, sendo 1...",12835,3,1,2,Alto,8602.150538


##OUTROS TRATAMENTOS
---

In [26]:
# Dropando a coluna Data Acidente
dados_listings.drop(['imovel_endereco_localizacao_coordinates',
                     'imovel_endereco_localizacao_type'], axis=1,inplace=True)

#**3. Loand**

##loand gcp

In [27]:
dados_listings.to_csv('gs://projeto_cat/TRATADOS/dados_listings-TRATADOS.csv', index=False)

##loand mysql

In [29]:
dados_listings['imovel_caracteristicas_propriedade'] = dados_listings['imovel_caracteristicas_propriedade'].apply(lambda x: str(x).replace("["," ").replace("]"," ").replace(" ", " ").replace("'", " "))
dados_listings['imovel_caracteristicas_condominio'] = dados_listings['imovel_caracteristicas_condominio'].apply(lambda x: str(x).replace("["," ").replace("]"," ").replace(" ", " ").replace("'", " "))
dados_listings['imovel_caracteristicas_entorno'] = dados_listings['imovel_caracteristicas_entorno'].apply(lambda x: str(x).replace("["," ").replace("]"," ").replace(" ", " ").replace("'", " "))

In [32]:
# dados de para realizar conexão com o servidor
servidor = '35.223.189.162'
nome_do_banco = 'DadosImoveis'
usuario = 'root'
senha = '123456'

# ciar uma SQLAlchemy engine para conectar com o MySQL 
engine = create_engine("mysql+pymysql://{user}:{pw}@{host}/{db}"
				.format(host=servidor, db=nome_do_banco, user=usuario, pw=senha))

# convertendo dataframe em uma table do sql                                   
dados_listings.to_sql('DadosImoveisTratados', engine, index=True, index_label="id")